In [2]:
scale = 3
almost_black = '#262626'
plt.rc('lines', linewidth=1.5)
plt.rc('axes', prop_cycle=cycler(color=Tableau_10.mpl_colors),
       labelsize=9*scale, edgecolor=almost_black)
plt.rc('xtick', labelsize=9*scale)
plt.rc('ytick', labelsize=9*scale)
plt.rc('legend', fontsize=8*scale)
plt.rc('font', family='serif', serif="Times New Roman", size=9*scale)
plt.rc('text', color=almost_black)
plt.rc('mathtext', fontset='stix')
plt.rc('xtick.major', size=10, width=1)
plt.rc('xtick.minor', size=5, width=1)
plt.rc('ytick.major', size=10, width=1)
plt.rc('ytick.minor', size=5, width=1)
plt.rc('figure', dpi=300, autolayout=True)

In [3]:
phi_1_files = [Path('pressure-traces') + p for p in [Path('Tc_700K_P0_2.1848_T0_353K_pressure.txt'), Path('Tc_734K_P0_1.8013_T0_353K_pressure.txt'), Path('Tc_775K_P0_1.4365_T0_353K_pressure.txt')]]
phi_2_files = [Path('pressure-traces') + p for p in [Path('Tc_699K_P0_0.7082_T0_373K_pressure.txt'), Path('Tc_723K_P0_0.7694_T0_373K_pressure.txt'), Path('Tc_786K_P0_0.7856_T0_413K_pressure.txt')]]

In [4]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('methyl-valerate-78135e2c2c7c.json',
                                                               scope)
gc = gspread.authorize(credentials)
ws = gc.open_by_key('1UFdGzEyx7PsKprPybi-lkbuWYS8BAS-cBm2IKSKY-UA').sheet1
recs = ws.get_all_records(empty2zero=True)
data = pd.DataFrame(recs)

phi_200_15_bar = data[(data['Equivalence Ratio'] == 2.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 15, rtol=1E-2))]
phi_100_30_bar = data[(data['Equivalence Ratio'] == 1.00) &
                      (np.isclose(data['Compressed Pressure (bar)'], 30, rtol=1E-2))]

phi_1_temps = phi_100_30_bar['Compressed Temperature (K)'].iloc[[1, 3, 5]]
phi_1_comp_times = phi_100_30_bar['Compression Time (ms)'].iloc[[1, 3, 5]]
phi_1_comp_times.iloc[-1] -= 1.0
phi_2_temps = phi_200_15_bar['Compressed Temperature (K)'].iloc[[3, 5, 11]]
phi_2_comp_times = phi_200_15_bar['Compression Time (ms)'].iloc[[3, 5, 11]]

In [5]:
fig, (ax_phi_1, ax_phi_2) = plt.subplots(ncols=2, figsize=[24, 9])

for i, f in enumerate(phi_1_files):
    data = np.loadtxt(str(f))
    ax_phi_1.plot(data[:, 0]*1000 - phi_1_comp_times.iloc[i], data[:, 1], label='$T_C = {}$ K'.format(phi_1_temps.iloc[i]))
    
nrdata = np.loadtxt('NR_Tc_700K_P0_2.1848_T0_353K_pressure.txt')
ax_phi_1.plot(nrdata[:, 0]*1000 + 0.25, nrdata[:, 1], label='Non-Reactive')

ax_phi_1.minorticks_on()

ax_phi_1.set_xlim(-20, 15)
ax_phi_1.set_ylim(0, 60)
ax_phi_1.set_ylabel('Pressure, bar')
ax_phi_1.set_xlabel('Time, ms')
ax_phi_1.legend(loc='upper center')
ax_phi_1.text(0.04, 0.9, 'a)', transform=ax_phi_1.transAxes, fontweight='bold')

In [6]:
for i, f in enumerate(phi_2_files):
    data = np.loadtxt(str(f))
    ax_phi_2.plot(data[:, 0]*1000 - phi_2_comp_times.iloc[i], data[:, 1], label='$T_C = {}$ K'.format(phi_2_temps.iloc[i]))

nrdata = np.loadtxt('NR_Tc_699K_P0_0.7082_T0_373K_pressure.txt')
ax_phi_2.plot(nrdata[:, 0]*1000, nrdata[:, 1], label='Non-Reactive')

ax_phi_2.minorticks_on()

ax_phi_2.set_xlim(-20, 40)
ax_phi_2.set_ylim(0, 25)
ax_phi_2.set_xlabel('Time, ms')
ax_phi_2.legend(loc='upper center')
ax_phi_2.text(0.04, 0.9, 'b)', transform=ax_phi_2.transAxes, fontweight='bold')

In [7]:
fig.savefig('pressure-traces.png')